In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Reshape, Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.metrics import confusion_matrix, classification_report
from IPython.display import display, HTML

Num GPUs Available:  1


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
df = pd.read_csv('/content/drive/MyDrive/WISDM_ar_v1.1_raw.csv')
df = df.dropna()

X = df[['acc_x', 'acc_y', 'acc_z']].apply(pd.to_numeric, errors='coerce').astype(np.float32)
y = df['activity']

y = pd.get_dummies(y)

TIME_PERIODS = 80
num_features = X.shape[1]
n_classes = y.shape[1]

def create_time_series(X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i: i + time_steps].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_time_series(X, y, TIME_PERIODS)

X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

print("Label yang tersedia:", list(y.columns))

Label yang tersedia: ['Downstairs', 'Jogging', 'Sitting', 'Standing', 'Upstairs', 'Walking']


In [11]:
model_m = Sequential()
# model_m.add(Reshape((TIME_PERIODS, num_features), input_shape=(TIME_PERIODS * num_features,)))
model_m.add(Dense(100, activation='relu', input_shape=(TIME_PERIODS, num_features)))
model_m.add(Dense(100, activation='relu'))
model_m.add(Dense(100, activation='relu'))
model_m.add(Flatten())
model_m.add(Dense(n_classes, activation='softmax'))

model_m.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 80, 100)             │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 80, 100)             │          10,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 80, 100)             │          10,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 8000)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 6)                   │          48,006 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 68,606 (267.99 KB)

 Trainable params: 68,606 (267.99 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model_m.compile(optimizer=Adam(learning_rate=0.001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_m.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32, callbacks=[early_stopping])

loss, accuracy = model_m.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Epoch 1/50
27451/27451 ━━━━━━━━━━━━━━━━━━━━ 104s 4ms/step - accuracy: 0.8691 - loss: 0.3856 - val_accuracy: 0.9243 - val_loss: 0.2154
Epoch 2/50
27451/27451 ━━━━━━━━━━━━━━━━━━━━ 98s 4ms/step - accuracy: 0.9335 - loss: 0.1966 - val_accuracy: 0.9410 - val_loss: 0.1778
Epoch 3/50
27451/27451 ━━━━━━━━━━━━━━━━━━━━ 142s 4ms/step - accuracy: 0.9443 - loss: 0.1661 - val_accuracy: 0.9480 - val_loss: 0.1581
Epoch 4/50
27451/27451 ━━━━━━━━━━━━━━━━━━━━ 101s 4ms/step - accuracy: 0.9501 - loss: 0.1498 - val_accuracy: 0.9555 - val_loss: 0.1340
Epoch 5/50
27451/27451 ━━━━━━━━━━━━━━━━━━━━ 140s 4ms/step - accuracy: 0.9535 - loss: 0.1390 - val_accuracy: 0.9535 - val_loss: 0.1412
Epoch 6/50
27451/27451 ━━━━━━━━━━━━━━━━━━━━ 100s 4ms/step - accuracy: 0.9568 - loss: 0.1316 - val_accuracy: 0.9540 - val_loss: 0.1447
Epoch 7/50
27451/27451 ━━━━━━━━━━━━━━━━━━━━ 94s 3ms/step - accuracy: 0.9587 - loss: 0.1239 - val_accuracy: 0.9547 - val_loss: 0.1418
Epoch 8/50
27451/27451 ━━━━━━━━━━━━━━━━━━━━ 101s 4ms/step - accu

In [23]:
validation_df = pd.read_csv('/content/valid.csv')
validation_df = validation_df.dropna()

X_val_pred = validation_df[['acc_x', 'acc_y', 'acc_z']].apply(pd.to_numeric, errors='coerce').astype(np.float32)
y_val_pred = validation_df['activity']

y_val_pred = pd.get_dummies(y_val_pred)

X_val_pred_seq, _ = create_time_series(X_val_pred, y_val_pred, TIME_PERIODS)

predictions = model_m.predict(X_val_pred_seq)

predicted_classes = np.argmax(predictions, axis=1)

class_names = list(y.columns)
predicted_activity_labels = [class_names[i] for i in predicted_classes]

print("Predictions on validation data:")
print(predicted_activity_labels)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predictions on validation data:
['Walking', 'Upstairs', 'Walking', 'Upstairs', 'Walking', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Sitting', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Standing', 'Standing', 'Standing', 'Standing', 'Standing', 'Standing', 'Standing', 'Standing', 'Standing']
